# Preparação dos Dados

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import arff

Importando os dados

In [3]:
file_path = "../data/raw/electricity-normalized.arff"
data, meta = arff.loadarff(file_path)

df = pd.DataFrame(data)

df

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
0,0.0000,b'2',0.000000,0.056443,0.439155,0.003467,0.422915,0.414912,b'UP'
1,0.0000,b'2',0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,b'UP'
2,0.0000,b'2',0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,b'UP'
3,0.0000,b'2',0.063830,0.045485,0.314639,0.003467,0.422915,0.414912,b'UP'
4,0.0000,b'2',0.085106,0.042482,0.251116,0.003467,0.422915,0.414912,b'DOWN'
...,...,...,...,...,...,...,...,...,...
45307,0.9158,b'7',0.914894,0.044224,0.340672,0.003033,0.255049,0.405263,b'DOWN'
45308,0.9158,b'7',0.936170,0.044884,0.355549,0.003072,0.241326,0.420614,b'DOWN'
45309,0.9158,b'7',0.957447,0.043593,0.340970,0.002983,0.247799,0.362281,b'DOWN'
45310,0.9158,b'7',0.978723,0.066651,0.329366,0.004630,0.345417,0.206579,b'UP'


Convertendo as colunas categóricas para Bytes de strings


In [4]:
for col in df.select_dtypes([object]):
    df[col] = df[col].str.decode("utf-8")

df

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
0,0.0000,2,0.000000,0.056443,0.439155,0.003467,0.422915,0.414912,UP
1,0.0000,2,0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,UP
2,0.0000,2,0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,UP
3,0.0000,2,0.063830,0.045485,0.314639,0.003467,0.422915,0.414912,UP
4,0.0000,2,0.085106,0.042482,0.251116,0.003467,0.422915,0.414912,DOWN
...,...,...,...,...,...,...,...,...,...
45307,0.9158,7,0.914894,0.044224,0.340672,0.003033,0.255049,0.405263,DOWN
45308,0.9158,7,0.936170,0.044884,0.355549,0.003072,0.241326,0.420614,DOWN
45309,0.9158,7,0.957447,0.043593,0.340970,0.002983,0.247799,0.362281,DOWN
45310,0.9158,7,0.978723,0.066651,0.329366,0.004630,0.345417,0.206579,UP


Encontrar o último valor sem medições

In [10]:
df.loc[df['vicprice'] != 0.003467].iloc[0]

date         0.442326
day                 1
period            0.0
nswprice     0.046325
nswdemand    0.298274
vicprice     0.003232
vicdemand    0.296737
transfer     0.500526
class            DOWN
Name: 17424, dtype: object

In [14]:
df = df.iloc[17424:]

In [15]:
df

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
17424,0.442326,1,0.000000,0.046325,0.298274,0.003232,0.296737,0.500526,DOWN
17425,0.442326,1,0.021277,0.045485,0.253794,0.003145,0.268255,0.500526,DOWN
17426,0.442326,1,0.042553,0.047316,0.231032,0.002910,0.306577,0.500526,DOWN
17427,0.442326,1,0.063830,0.036658,0.183130,0.002956,0.271621,0.500526,DOWN
17428,0.442326,1,0.085106,0.041882,0.148319,0.002659,0.219834,0.500526,DOWN
...,...,...,...,...,...,...,...,...,...
45307,0.915800,7,0.914894,0.044224,0.340672,0.003033,0.255049,0.405263,DOWN
45308,0.915800,7,0.936170,0.044884,0.355549,0.003072,0.241326,0.420614,DOWN
45309,0.915800,7,0.957447,0.043593,0.340970,0.002983,0.247799,0.362281,DOWN
45310,0.915800,7,0.978723,0.066651,0.329366,0.004630,0.345417,0.206579,UP


Primeiro precisamos retirar os dados que não houveram medições de victoria e transfer

## 1. Seleção dos Dados


### 1.1 Dados a Incluir

### 1.2 Dados a Excluir

## 2. Limpeza dos Dados

## 3. Construção de Dados

### 3.1 Atributos Derivados

## 4. Integração de Dados

## 5. Formatação de Dados 